# ANN 신경망 알고리즘

- 여러 층을 거치면서 정보가 연속된 Filter 통과, 의미있는 정보로 추출
- Deep Learning : 신경망 모델(ANN, Artifical Neural Network)을 활용해 표현층 (Layer, 연속된 Filter 층) 정보를 학습
- 딥러닝 신경망 알고리즘은 특성공학의 과정을 자동화 한다
  - 따로 특성공학 기법을 구분하여 적용하지 않는다, 즉 Layer에서 데이터의 Edge 특징과 규칙을 찾아내는 형태)
- 신경망 알고리즘의 장점
  - 신경망 알고리즘은 거의 모든 함수를 추정할 수 있다 (보편 근사의 정리) -> 매우 유연한 확정성이 있는 알고리즘
  - 기존과 다른 형태/ 다른 형식의 데이터에 대해서도 잘 학습

- Node : 데이터를 처리하는 활성 함수 (Activation Function) / 데이터를 처리하는 하나의 함수 식
- Layer : 같은 동작을 수행하는 Node들이 모여있는 필터 층
- Weight : 입력받은 정보를 어떻게 처리해야할지에 대한 처리 정보 (Layer Parameter)
- 신경망 알고리즘에서 학습 : 예측을 정확하게 하기 위해, Layer 내 있는 Weight를 적절하게 찾아 주는 것
- 손실 함수 (Cross Function, 비용함수)
  - 예측값과 실제 데이터값의 차이를 계산하는 함수
  - Layer 내 Weight값을 Update 결정하는 역할
  - 보통 SSE(Sum of Square Error) 계산 : Σ(yᵢ - ŷᵢ)²

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stasts

In [29]:
from sklearn.model_selection import train_test_split # 학습 데이터 검증 데이터 분할
from imblearn.pipeline import make_pipeline # 특성공학 기법을 적용할 파이프라인을 구성
from sklearn.compose import make_column_transformer # 문자와 숫자데이터 병렬로 파이프 구성
from sklearn.impute import KNNImputer # KNN 결측값 대치 기법 (알고리즘을 이용한 대치)
from sklearn.impute import SimpleImputer # 범주형 데이터 결측 처리 (최빈값)
from sklearn.preprocessing import OneHotEncoder # 문자 -> 숫자 (인코딩)
from sklearn.preprocessing import MinMaxScaler # 숫자 -> 숫자 (스케일링)
from sklearn.model_selection import GridSearchCV # 하이퍼파리미텨 튜닝 + 교차검증
from imblearn.combine import SMOTETomek # 불균형 데이터 처리
from sklearn.feature_selection import SelectKBest, f_classif # 변수 선택법
from sklearn.metrics import classification_report # 분류모델에 대한 성능 평가 실시

In [6]:
df_person = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\22_Data.csv')
print(df_person.shape)
df_person.head(2)

(100233, 10)


,Unnamed: 0,고객ID,연령,성별,직업,추정소득,가구추정소득,가족원수,자녀평균연령,배우자직업
0,0,1,50,1,공무원,5400,7700,4,24.0,주부
1,1,2,50,1,자영업,5500,8100,4,29.0,주부


In [9]:
df_loan = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\23_Data.csv')
print(df_loan.shape)
df_loan.head(2)

(100233, 17)


,Unnamed: 0,고객ID,대출건수_1금융,대출건수_2금융,대출건수_3금융권,대출건수_기타,대출액_1금융,대출액_2금융,대출액_3금융,대출액_기타,유지기간_1금융,유지기간_2금융,신용카드발급수,신용카드유지기간,보증건수,보증금액,대출연체여부
0,0,1,1,0,0,0,9001,9001,9001,0,1,0,2,13,3,420001,0
1,1,2,1,0,0,0,24001,0,24001,0,0,0,2,121,0,0,0


In [16]:
df1 = pd.merge(df_person, df_loan, on='고객ID', how='inner')

In [17]:
df1['연령'].unique()

array(['50', '60', '35', '45', '40', '55', '25', '30', '*', '65', '20',
       '70'], dtype=object)

In [18]:
df1['연령_clean'] = df1['연령'].replace('*', np.nan).astype(float) # np.nan 결측값이 데이터 속성이 -> float 실수형태

In [21]:
df1['연령_clean'].unique()

array([50., 60., 35., 45., 40., 55., 25., 30., nan, 65., 20., 70.])

In [22]:
df2 = df1.drop(columns='배우자직업').dropna()

In [24]:
# 고객들의 금융관련 정보를 입력했을 때, 대출이 연체될 고객인지 아닌지 분류하는 분류모델 ANN생성
X = df2[['연령_clean', '추정소득', '가구추정소득', '가족원수','자녀평균연령', '대출건수_1금융', 
         '대출건수_2금융', '대출건수_3금융권','대출건수_기타', '대출액_1금융', '대출액_2금융', '대출액_3금융', 
          '대출액_기타', '유지기간_1금융','유지기간_2금융', '신용카드발급수','신용카드유지기간', '보증건수', '보증금액']]
Y = df2['대출연체여부']

In [34]:
X_train, X_test, Y_train , Y_test = train_test_split(X,Y, random_state=1234)

In [ ]:
!pip uninstall keras
!pip3 install keras --upgrade

In [28]:
!pip install tensorflow

   ---------------------------------------- 0.0/377.0 MB ? eta -:--:--
   ---------------------------------------- 0.7/377.0 MB 15.7 MB/s eta 0:00:24
   ---------------------------------------- 1.7/377.0 MB 22.2 MB/s eta 0:00:17
   ---------------------------------------- 2.9/377.0 MB 26.6 MB/s eta 0:00:15
   ---------------------------------------- 4.3/377.0 MB 27.5 MB/s eta 0:00:14
    --------------------------------------- 6.0/377.0 MB 29.4 MB/s eta 0:00:13
    --------------------------------------- 7.5/377.0 MB 31.8 MB/s eta 0:00:12
    --------------------------------------- 9.3/377.0 MB 33.0 MB/s eta 0:00:12
   - -------------------------------------- 10.5/377.0 MB 34.4 MB/s eta 0:00:11
   - -------------------------------------- 12.6/377.0 MB 38.6 MB/s eta 0:00:10
   - -------------------------------------- 14.4/377.0 MB 40.9 MB/s eta 0:00:09
   - -------------------------------------- 16.5/377.0 MB 46.9 MB/s eta 0:00:08
   - -------------------------------------- 17.6/377.0 M

In [44]:
import tensorflow.keras as keras
keras.__version__

'3.3.3'

In [45]:
from keras import models
from keras.layers import Dense

In [ ]:
import numpy as np
backend.convert_to_numpy = np.array

In [46]:
model = keras.models.Sequential() # 신경망을 구성하는데 있어 가장 기번족인 전결함(순차적으로 Layer를 통과하여 데이터가 처리되는 구조)
# Dense : Layer 층을 구성 / 16개의 Node로 구성 / 활성 함수 relu
model.add( (Dense(16, activation='relu', input_shape=(19,))) ) # 첫번째 층 (16개 Node) / 데이터가 들어오는 층
model.add(Dense(16, activation='relu')) # 두번째 층 구성 (16게 Node)
# 최종 결과를 하나의 Node로 출력 -> Sigmoid 0~1 사이의 값이 출력되는 함수로 결과를 계산
model.add(Dense(1, activation='sigmoid')) 
# 각 Node에 있는 Weight를 Update 하기 위한 전략을 구성
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# binary_crossentropy 라는 손실함수를 이용하여, 실제값과 신경망 알고리즘의 예측 결과값이 차이가 있는지 평가 (평가 지표 - 정확도)
# optimizer='rmsprop' 최적화 기법을 이용해, 앞서 Node에 있는 Weight를 Update 실시
result = model.fit(X_train, Y_train, epochs=20, validation_split=0.3)

Epoch 1/20


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'